In [1]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.2,
    num_predict=300     
)

E:\Temp\ipykernel_25976\4048587293.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [3]:
response = llm.invoke('عرف عن نفسك')
print(response.content)

أنا بروتوكولًا مصممًا بواسطة شركة Meta، صُمم لتقديم المعلومات والاستجابة السريعة. يمكنني أن أتعلم مع الوقت وبالتالي أصبحت أكثر تعقيدًا. يتم تحديثي دورياً لضمان دقتها. إنه عمل جهد كبير من قبل فريق كبير من المصممين والمتخصصين في الذكاء الاصطناعي.


## faiss DB

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
import numpy as np
import os


def normalize(vectors):
    return [v / np.linalg.norm(v) if np.linalg.norm(v) != 0 else v for v in vectors]


folder = r"./transcriptions"
docs = []
for fn in os.listdir(folder):
    if fn.endswith(".txt"):
        for d in TextLoader(os.path.join(folder, fn), encoding="utf-8").load():
            d.metadata = {"source": fn}
            docs.append(d)

embedding = OllamaEmbeddings(model="nomic-embed-text")

raw_embeddings = embedding.embed_documents([doc.page_content for doc in docs])
normalized_embeddings = normalize(raw_embeddings)

text_embedding_pairs = [
    (doc.page_content, embedding_vec)
    for doc, embedding_vec in zip(docs, normalized_embeddings)
]

vectordb = FAISS.from_embeddings(text_embedding_pairs, embedding)

vectordb.save_local(r"./faiss_index")

E:\Temp\ipykernel_25976\4221662540.py:21: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")


In [3]:
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  
)

In [ ]:
import numpy as np

def normalize(vectors):
    return [v / np.linalg.norm(v) if np.linalg.norm(v) != 0 else v for v in vectors]

def query_faiss(question: str, k: int = 3, threshold: float = 0.7):
    query_embedding = embedding.embed_query(question)
    normalized_query = normalize([query_embedding])[0]

    docs_and_scores = vectordb.similarity_search_with_score_by_vector(normalized_query, k=k)

    results = []
    for doc, score in docs_and_scores:
        results.append({
            "text": doc.page_content,
            "metadata": doc.metadata,
            "score": score 
        })

    filtered = [r for r in results if r["score"] >= threshold]

    return filtered

In [5]:
query = 'ما هي البروتينات المضاضه للكائنات الدقيقه'
results = query_faiss(query, k=5, threshold=0.6)
for i, r in enumerate(results, 1):
    print(f"{i}. cosine = {r['score']:.4f} | Source: {r['metadata'].get('source')}")
    print(r['text'][:100], "\n---\n")

1. cosine = 0.6063 | Source: None
بروتينات. خلي بالك اوي من كلمة بروتينات دي. مضادة. للكائنات. الدقيق. تانية كان اسمها مواد كيميائية م 
---

2. cosine = 0.6134 | Source: None
المضادة للكائنات الدقيقة. احنا مش عايزين الكلام المعقد الكبير. المواد الكيميائية المضادة للكائنات ال 
---

3. cosine = 0.6175 | Source: None
بروتينات مضادة للكائنات. اتضقق. ازا انت كده يعتبر خلاص خلاصنا. يبقى ثانية اولا احنا دلوقتي بنتكلم في 
---

4. cosine = 0.6268 | Source: None
تاني. مستقبلات كده كده موجودة. المواد الكيميائية المساعدة او المواد الكيميائية يعني اللي هي مضادة ال 
---

5. cosine = 0.6566 | Source: None
البروتينات المضادة للكائنات الضقيقة. طيب تعالى بقى كده واحدة واحدة ونشوف المستقبلات. المستقبلات يعني 
---



## Conversational Retrieval Chain

In [6]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)

condense_template = PromptTemplate.from_template(
    "Given the conversation below, rewrite the user's last question as a standalone question. "
    "Important: write it in the SAME language as the user asked.\n\n"
    "Conversation:\n{chat_history}\n\n"
    "Standalone question:"
)

memory = ConversationSummaryBufferMemory(#history 
    llm=llm,                  
    memory_key="chat_history", 
    return_messages=True,     
    buffer_size=3,          
    max_token_limit=1000
)

qa_prompt = PromptTemplate.from_template(
    """
    You are a helpful tutor for school students. 
    Always explain in a simple way that a child can understand. 
    Use clear examples, and avoid difficult academic words.

    Context:
    {context}

    Question:
    {question}

    Answer in the same language of the question:
    """
)

qa_chain_stuff = StuffDocumentsChain(
    llm_chain=LLMChain(llm=llm, prompt=qa_prompt),
    document_variable_name="context"
)


from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    combine_docs_chain=qa_chain_stuff,
    question_generator=LLMChain(llm=llm, prompt=condense_template),
    memory=memory,
    return_source_documents=False,
    verbose=False,
    max_tokens_limit=300,
    output_key="answer"  # ← ده المفتاح اللي هيتخزن في الذاكرة
)


E:\Temp\ipykernel_25976\1833664569.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(#history
E:\Temp\ipykernel_25976\1833664569.py:39: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=llm, prompt=qa_prompt),
E:\Temp\ipykernel_25976\1833664569.py:38: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  qa_chain_stuff = StuffDocumentsChain(
E:\Temp\ipykernel_25976\1833664569.py:46: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~create_history_aware_retriev

In [34]:
print(memory.load_memory_variables({})["chat_history"][0])
print(memory.load_memory_variables({})["chat_history"][1])
print(memory.load_memory_variables({})["chat_history"][2])
print(memory.load_memory_variables({})["chat_history"][3])
print(memory.load_memory_variables({})["chat_history"][4])
print(memory.load_memory_variables({})["chat_history"][5])


content='I\'ll do my best to translate the new line of conversation.\n\nCurrent summary:\nThe human asks about proteins that fight against microscopic organisms. The AI explains that these proteins are a necessary substance, and they work to kill microscopic entities like bacteria and viruses. For example, when you feel itchy after using the bathroom, it might be due to your body producing proteins that fight off bacteria causing the itch.\n\nNew lines of conversation:\nHuman: ما هي البروتينات المضاضه للكائنات الدقيقه (What are the proteins that fight against microscopic organisms?)\n\nNew summary:\nThe human asks about proteins that fight against microscopic organisms. The AI explains that these proteins are a necessary substance, and they work to kill microscopic entities like bacteria and viruses. For example, when you feel itchy after using the bathroom, it might be due to your body producing proteins that fight off bacteria causing the itch.\n\n(Note: I translated "ما هي البروتينا

In [35]:
def first_reply_from_vector(question: str, chat_history=None, top_k_sources=3):
    search = query_faiss(question, top_k_sources, threshold=0.7)
    if not search:
        return None

    inputs = {"question": question}
    if chat_history:
        inputs["chat_history"] = chat_history

    result = qa_chain(inputs)

    answer = result.get("answer") or result.get("result") or result.get("output_text") or ""

    sources = []
    for item in search[:top_k_sources]:
        sources.append({
            "text": item["text"],
            "metadata": item.get("metadata", {})
        })

    return {
        "source": "video",
        "answer": answer.strip(),
        "sources": sources
    }

## serch from internet

In [ ]:
from langchain.schema import Document

def qa_chain_web(question: str, web_results: list, chat_history=None):
    docs = []
    links = []

    for r in web_results:
        body = r.get("body", "")
        href = r.get("href", "")

        if body.strip() and href.strip():
            docs.append(Document(page_content=body.strip()))
            links.append(href.strip())

    if not docs:
        return {
            "source": "internet",
            "answer": "لا توجد نتائج من الإنترنت.",
            "sources": []
        }

    if chat_history:
        memory.save_context({"input": question}, {"output": ""})

    answer = qa_chain_stuff.run({
        "input_documents": docs,
        "question": question
    })

    return {
        "source": "internet",
        "answer": answer.strip(),
        "sources": links  # ← روابط فقط
    }

In [ ]:
from langchain.embeddings import OllamaEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from ddgs import DDGS

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

embedding_function = OllamaEmbeddings(model="nomic-embed-text")

def rerank_web_results(question, max_results=5, threshold=0.65):
    with DDGS() as ddgs:
        results = list(ddgs.text(question, max_results=max_results))

    if not results:
        return []

    q_emb = normalize(embedding_function.embed_query(question))
    filtered = []

    for r in results:
        body = r.get("body", "")
        href = r.get("href", "")

        if not body.strip() or not href:
            continue

        try:
            d_emb = normalize(embedding_function.embed_query(body))
            sim = cosine_similarity(
                np.array(q_emb).reshape(1, -1),
                np.array(d_emb).reshape(1, -1)
            )[0][0]

            if sim >= threshold:
                filtered.append({
                    "body": body.strip(),
                    "href": href.strip()
                })

        except Exception:
            continue

    return filtered

## the final chat-with-video

In [33]:
q = 'ما هي البروتينات المضاضه للكائنات الدقيقه'
out = first_reply_from_vector(q)
if out is None:
    print("answer from web")
    results = rerank_web_results(q,5)
    if not results:
        print('no ans')
    else:
        web_result = qa_chain_web(q,results,memory)##from net and llm
        print(web_result['answer'])
        print(web_result['sources'])
else:
    print('answer from video')
    print(out["answer"])
    print('*'*50)

answer from web
حسناً يا طالب! سأشرح لك هذا الجملة بسهولة.

الجملة تقول: "ما هي البروتينات المضادة للكائنات الدقيقة؟"

هذا يعني أننا نريد معرفة ما هو نوع من البروتينات التي تعمل ضد الكائنات الدقيقة، مثل البكتيريا والفيروسات.

باختصار، البروتينات المضادة للكائنات الدقيقة هي نوع من البروتينات التي تحمي الجسم من العدوى والمرض.

فهمت جيداً؟
['https://mawdoo3.com/أنواع_ما_في_اللغة_العربية', 'https://loghate.com/s/ما-هي-أنواع-ما-وإعرابها']


In [ ]:
# Python 3.12.7 environment with all requirements installed successfully!
import sys
print(f"Python version: {sys.version}")
print("All packages from requirements.txt are installed and ready to use!")

In [36]:
%%writefile app1.py
import streamlit as st
import time

# --- Page setup ---
st.set_page_config(page_title="Video Chat", page_icon="🎥", layout="wide")

st.markdown(
    """
    <style>
    .block-container {padding-top: 1rem; padding-bottom: 1rem;}
    .stChatInput {position: fixed; bottom: 20px; right: 2%; width: 28% !important;}
    .stChatMessage {max-width: 90%;}
    </style>
    """,
    unsafe_allow_html=True,
)

# --- Initialize session state ---
if "messages" not in st.session_state:
    st.session_state.messages = []
if "processing" not in st.session_state:
    st.session_state.processing = False

# --- MAIN SECTION ---
st.header("🎥 Video Chat")

# --- Sidebar with clear chat button ---
with st.sidebar:
    st.header("🎥 Video Chat")
    st.write(f"عدد الرسائل: {len(st.session_state.messages)}")
    if st.button("🗑 مسح المحادثة"):
        st.session_state.messages = []
        st.success("تم مسح المحادثة!")
        st.rerun()

# --- Layout ---
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("تشغيل الفيديو")
    st.video("https://youtu.be/-zHgacHhS5Q")
    st.write("فيديو تجريبي يعمل تلقائيًا")

with col2:
    st.subheader("💬 المحادثة الحية")
    chat_container = st.container()
    with chat_container:
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

# --- Chat input ---
if not st.session_state.processing:
    if prompt := st.chat_input("اكتب سؤالك عن الفيديو أو الموضوع..."):
        st.session_state.processing = True
        st.session_state.messages.append({"role": "user", "content": prompt})

        with chat_container:
            with st.chat_message("user"):
                st.markdown(prompt)

        # --- الرد من الفيديو أو الإنترنت ---
        q = prompt
        out = first_reply_from_vector(q)

        if out is None:
            results = rerank_web_results(q, 5)
            if not results:
                assistant_response = "❌ لم أجد إجابة مناسبة من الإنترنت."
            else:
                web_result = qa_chain_web(q, results, memory)
                answer = web_result['answer']
                sources = web_result['sources']

                assistant_response = answer.strip() + "\n\n📚 المصادر:\n"
                for i, link in enumerate(sources, 1):
                    assistant_response += f"{i}. [رابط المصدر]({link})\n"
        else:
            assistant_response = out["answer"].strip() + "\n\n✅ الإجابة من محتوى الفيديو."

        # --- عرض الرد ---
        with chat_container:
            with st.chat_message("assistant"):
                message_placeholder = st.empty()
                full_response = ""
                for chunk in assistant_response.split():
                    full_response += chunk + " "
                    time.sleep(0.05)
                    message_placeholder.markdown(full_response + "▌")
                message_placeholder.markdown(full_response)

        st.session_state.messages.append({"role": "assistant", "content": assistant_response})
        st.session_state.processing = False
        st.rerun()

# --- Footer ---
st.divider()


Writing app1.py


'streamlet' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
! streamlit run app1.py

^C
